## Question 3: Using Python 3, implement a Facebook and/or Instagram connector to collect posts (images, text, and comments related to the images) based on a defined topic, for example, "the death of President Jacques Chirac." It is recommended to store the text and images in a MongoDB database. The goal of this question is to evaluate the quality of your source code

#### To approach this question, I have to 
#### 1. Create a Meta Graph API App. This app serves as a API connector which provides both user and page access tokens.
#### 2. Once this app is created, I have to set up a MongoDB Atlas account which provides the database URI that can be connected in the code and later used to store the data.
#### 3. Install and import necessary libraries that allow interact with the Facebook API and MongoDB database like `GraphAPI`, `pymongo`, and `MongoClient`

In [57]:
import requests
import urllib3
from pyfacebook import GraphAPI
import pymongo
from pymongo import MongoClient

In [89]:
# Declaring Constants
ACCESS_TOKEN =  'EAAI6kA3lzZAwBO7GtHkwYykMSVxnUzAsma6oCYaSp5p7s2CRSZARXYNmUkKqG7XrizSLZCvIyk1YUhDk5JTuA4hgGTvWruXZBuP83SQUdWZBLHDqRELEytgtqzNFH2NI2DK4Y4IHu5ji77IMozYzSivnqRCKNYPpG2FtCIbE1zZAYXEdgLQARRFQu2A11HcCz1DqGW00KYJrzBYtjgXwZDZD' 
API_URL = 'https://graph.facebook.com/v21.0/2895093670650302?access_token=ACCESS-TOKEN'
TOPIC = 'Trump'
DB_NAME = 'social_media'
COLLECTION_NAME = 'posts'

##### Here, I connect the MongoDB database to the code using the URI provided by MongoDB.

In [90]:
# MongoDB connection
client = MongoClient('mongodb+srv://mikeaagna:<db_password>@cluster0.ukq4r.mongodb.net/?retryWrites=true&w=majority&appName=Cluster0')
collection = db[COLLECTION_NAME]

##### I define a function here which takes the keyword and limit of posts containing the keyword as its arguments, this function block of this code searches for posts containing the keyword `Trump`.

In [91]:
def fetch_facebook_posts(topic, limit=10):

    search_url = f"{API_URL}/search"
    search_params = {
        'q': topic,  
        'type': 'post',
        'fields': 'id,name,picture',
        'access_token': ACCESS_TOKEN,
        'limit': limit
    }
    response = requests.get(search_url, params=search_params)
    if response.status_code != 200:
        raise Exception(f"Error fetching posts: {response.json()}")
    
    return response.json().get('data', [])

##### Here again, I define another function to store the results in the database I have already created.

In [92]:
def store_in_mongodb(posts):
    
    for post in posts:
        # Parsing the required fields
        post_id = post.get('id')
        message = post.get('message', 'No message provided')
        created_time = post.get('created_time')
        comments = post.get('comments', {}).get('data', [])
        attachments = post.get('attachments', {}).get('data', [])
        
        #to fetch images if any 
        images = []
        for attachment in attachments:
            if 'media' in attachment:
                images.append(attachment['media'].get('image', {}).get('src'))

        
        post_document = {
            'post_id': post_id,
            'message': message,
            'created_time': created_time,
            'comments': [{'text': c['message'], 'user': c['from']['name']} for c in comments],
            'images': images
        }

        #to insert into mongodb
        collection.insert_one(post_document)
        print(f"Inserted post {post_id} into MongoDB.")

##### Finally, I update the main function which validates previous processes.
##### The success of the code is heavily dependent on the Meta permissions and access. I have checked the logic of my code more than 10 times and the only error I have encountered is with the permission to access posts by Facebook. 
##### `NOTE:` It requires a different `GET` process to fetch post videos as this has been updated by Facebook.
##### While working on this test, I observed that the permissions for Facebook API have changed and hence certain posts cannot be accessed.

In [93]:
def main():
    try:
        print(f"Fetching posts related to '{TOPIC}'...")
        posts = fetch_facebook_posts(TOPIC, limit=10)
        print(f"Fetched {len(posts)} posts. Storing in MongoDB...")
        store_in_mongodb(posts)
        print("All posts have been stored successfully.")
    except Exception as e:
        print(f"An error occurred: {e}")

if __name__ == "__main__":
    main()

Fetching posts related to 'Trump'...
Fetched 0 posts. Storing in MongoDB...
All posts have been stored successfully.
